# 自定义层

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf

## 网络层layer的常见操作

通常机器学习模型可以表示为简单网络层的堆叠与组合，而tensorflow就提供了常见的网络层，为我们编写神经网络程序提供了便利。 TensorFlow2推荐使用tf.keras来构建网络层，tf.keras来自原生keras，用其来构建网络具有更好的可读性和易用性。

In [4]:
layer = tf.keras.layers.Dense(100)

# 也可以添加输入维度限制
layer = tf.keras.layers.Dense(100, input_shape=(None, 20))

可以在文档中查看预先存在的图层的完整列表。 它包括Dense，Conv2D，LSTM，BatchNormalization，Dropout等等。

每个层都可以当作一个函数，然后以输入的数据作为函数的输入

In [6]:
layer(tf.ones([6, 6]))

print(layer.variables) # 包含了权重和偏置

[<tf.Variable 'dense_5/kernel:0' shape=(6, 100) dtype=float32, numpy=
array([[-1.96505249e-01,  1.44933507e-01, -4.31172103e-02,
        -4.42382395e-02,  2.09344998e-01,  5.82718998e-02,
        -1.15520120e-01,  1.01626366e-02,  1.23267189e-01,
        -8.65449309e-02,  2.16348320e-02, -6.33617640e-02,
         1.21080622e-01, -7.96761066e-02, -2.25779146e-01,
        -1.75690621e-01, -2.35477209e-01, -1.31327868e-01,
         1.80480704e-01, -1.51055560e-01, -8.35041553e-02,
        -9.08717364e-02,  1.52834073e-01, -1.11705408e-01,
         1.71547547e-01, -1.53688997e-01, -1.53948277e-01,
         1.08774006e-02, -3.43349427e-02,  1.88626498e-02,
        -8.72506201e-02, -2.15675786e-01, -7.10344017e-02,
        -2.16574177e-01, -6.62512481e-02,  1.49431333e-01,
        -1.04028687e-01,  3.32798809e-02,  1.57843724e-01,
         7.09565133e-02,  7.12100714e-02,  4.90203053e-02,
        -5.35197854e-02,  1.58766255e-01, -1.86012253e-01,
         1.88821107e-02, -4.89718616e-02,  1.

In [7]:
print(layer.kernel, layer.bias)  # 也可以分别取出权重和偏置

<tf.Variable 'dense_5/kernel:0' shape=(6, 100) dtype=float32, numpy=
array([[-1.96505249e-01,  1.44933507e-01, -4.31172103e-02,
        -4.42382395e-02,  2.09344998e-01,  5.82718998e-02,
        -1.15520120e-01,  1.01626366e-02,  1.23267189e-01,
        -8.65449309e-02,  2.16348320e-02, -6.33617640e-02,
         1.21080622e-01, -7.96761066e-02, -2.25779146e-01,
        -1.75690621e-01, -2.35477209e-01, -1.31327868e-01,
         1.80480704e-01, -1.51055560e-01, -8.35041553e-02,
        -9.08717364e-02,  1.52834073e-01, -1.11705408e-01,
         1.71547547e-01, -1.53688997e-01, -1.53948277e-01,
         1.08774006e-02, -3.43349427e-02,  1.88626498e-02,
        -8.72506201e-02, -2.15675786e-01, -7.10344017e-02,
        -2.16574177e-01, -6.62512481e-02,  1.49431333e-01,
        -1.04028687e-01,  3.32798809e-02,  1.57843724e-01,
         7.09565133e-02,  7.12100714e-02,  4.90203053e-02,
        -5.35197854e-02,  1.58766255e-01, -1.86012253e-01,
         1.88821107e-02, -4.89718616e-02,  1.8

## 实现自定义网络层

实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：

- ```__init__()```函数，你可以在其中执行所有与输入无关的初始化
- ```build()```函数，可以获得输入张量的形状，并可以进行其余的初始化
- ```call()```函数，构建网络结构，进行前向传播

实际上，你不必等到调用```build()```来创建网络结构，您也可以在 ```__init__()``` 中创建它们。 但是，在```build()```中创建它们的优点是它可以根据图层将要操作的输入的形状启用后期的网络构建。 另一方面，在 ```__init__``` 中创建变量意味着需要明确指定创建变量所需的形状。

In [8]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, n_outputs):
        super(MyDense, self).__init__()
        self.n_outputs = n_outputs

    def build(self, input_shape):
        self.kernel = self.add_variable('kernel',
                                       shape=[int(input_shape[-1]),
                                             self.n_outputs])
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = MyDense(10)
print(layer(tf.ones([6, 5])))
print(layer.trainable_variables)

W0924 14:21:49.657814 140735707075456 deprecation.py:323] From <ipython-input-8-1e113d7ca156>:9: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


tf.Tensor(
[[ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]
 [ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]
 [ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]
 [ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]
 [ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]
 [ 1.0790207  -0.56067765 -0.23647422 -0.64915043  0.2764306   1.1679603
   0.7331519   0.12444073  0.616171   -0.22489607]], shape=(6, 10), dtype=float32)
[<tf.Variable 'my_dense/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.54448   ,  0.00800312,  0.05057257, -0.4000849 ,  0.31148863,
         0.4172247 ,  0.45564395, -0.28062525,  0.27774274, -0.3903533

## 网络层组合

机器学习模型中有很多是通过叠加不同的结构层组合而成的，如resnet的每个残差块就是“卷积+批标准化+残差连接”的组合。

在tensorflow2中要创建一个包含多个网络层的的结构，一般继承与tf.keras.Model类。

In [9]:
# 残差块
class ResnetBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetBlock, self).__init__(name='resnet_block')

        # 每个子层卷积核数
        filter1, filter2, filter3 = filters

        # 三个子层，每层1个卷积加一个批正则化
        # 第一个子层， 1*1的卷积
        self.conv1 = tf.keras.layers.Conv2D(filter1, (1,1))
        self.bn1 = tf.keras.layers.BatchNormalization()
        # 第二个子层， 使用特点的kernel_size
        self.conv2 = tf.keras.layers.Conv2D(filter2, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        # 第三个子层，1*1卷积
        self.conv3 = tf.keras.layers.Conv2D(filter3, (1,1))
        self.bn3 = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=False):

        # 堆叠每个子层
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        # 残差连接
        x += inputs
        outputs = tf.nn.relu(x)

        return outputs

resnetBlock = ResnetBlock(2, [6,4,9])
# 数据测试
print(resnetBlock(tf.ones([1,3,9,9])))
# 查看网络中的变量名
print([x.name for x in resnetBlock.trainable_variables])

tf.Tensor(
[[[[1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.4239442  0.9072625  0.4803856  0.         1.0187587  1.7749279
    1.7444522  1.0845194  1.2420487 ]
   [1.2715074  0.65333825 0.19249606 0.         1.5315349  1.6433787
    1.2518395  0.7032629  1.4371998 ]]

  [[1.4239442  0.9072625